Student: Arne Cools - IAI3 - 2022/2023

# Model Evaluation & Hyperparameter Tuning
## Part 1 - Dataset & Data Preprocessing
### 1.2 Data Preprocessing

#### 1.2.1 Data Preparation
Clean the data, remove outliers, handle missing values, etc.

In [93]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, SelectPercentile
from sklearn.preprocessing import LabelEncoder

In [94]:
apps = pd.read_csv('../googleplaystore.csv')
apps.head(50)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


Let's start with converting Data Types:
* Convert the "Reviews" column to numeric (integer) data type.
* Convert the "Size" column to a numeric format (e.g., megabytes) for easier handling.
* Convert the "Installs" column to numeric data type, removing the '+' and ',' characters.
* Convert the "Price" column to numeric, removing the '$' sign.
* Convert the "Last Updated" column to a datetime data type.

In [95]:
apps = pd.read_csv('../googleplaystore.csv')
apps['Reviews'] = pd.to_numeric(apps['Reviews'], errors='coerce')
apps['Size'] = apps['Size'].apply(lambda x: float(x.replace('k','')) / 1024 if 'k' in str(x) else x)  # Convert k to M
apps['Size'] = apps['Size'].apply(lambda x: None if x == 'Varies with device' else x) # Handles "Varies with device" values
apps['Size'] = apps['Size'].str.replace('M', '').str.replace('k', '').astype(float)
apps['Installs'] = apps['Installs'].str.replace('+', '').str.replace(',', '').astype('int64')
apps['Price'] = apps['Price'].str.replace('$', '').astype(float)
apps['Last Updated'] = pd.to_datetime(apps['Last Updated'])
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10840 entries, 0 to 10839
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   App             10840 non-null  object        
 1   Category        10840 non-null  object        
 2   Rating          9366 non-null   float64       
 3   Reviews         10840 non-null  int64         
 4   Size            8829 non-null   float64       
 5   Installs        10840 non-null  int64         
 6   Type            10839 non-null  object        
 7   Price           10840 non-null  float64       
 8   Content Rating  10840 non-null  object        
 9   Genres          10840 non-null  object        
 10  Last Updated    10840 non-null  datetime64[ns]
 11  Current Ver     10832 non-null  object        
 12  Android Ver     10838 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(7)
memory usage: 1.1+ MB


Some of the most popular apps are represented more than once but with the same data. We will remove the duplicates since these might affect the results of our model.

In [96]:
print(f"Shape before removing duplicates {pd.DataFrame(apps).shape}")
apps = apps.drop_duplicates(subset='App', keep='first')
print(f"Shape after removing duplicates {pd.DataFrame(apps).shape}")

Shape before removing duplicates (10840, 13)
Shape after removing duplicates (9659, 13)


Removing outliers in the dataset.
To detect outliers I will use the z-score approach. This code removes all rows with a column value more than 3 standard deviations away from the mean.

In [97]:
print(f"Shape before removing outliers {pd.DataFrame(apps).shape}")

def remove_outliers(df,columns,n_std):
    for col in columns:

        mean = df[col].mean()
        sd = df[col].std()

        df = df[(df[col] <= mean+(n_std*sd))]

    return df

apps = remove_outliers(apps, ['Price'], 3)

print(f"Shape after removing outliers {pd.DataFrame(apps).shape}")


Shape before removing outliers (9659, 13)
Shape after removing outliers (9636, 13)


Now we check for NA values

In [98]:
pd.DataFrame(apps.isna().sum(), columns=['# of missing values'])

,# of missing values
App,0
Category,0
Rating,1456
Reviews,0
Size,1540
Installs,0
Type,1
Price,0
Content Rating,0
Genres,0


The NA values of Size are expected, these are values that varies per device type. We convert them to 0 here. The rows with NA values for rating will be deleted, since this will be an important parameter for our model.

In [99]:
apps = apps.dropna(subset=['Rating'])
apps['Size'] = apps['Size'].fillna(0)

apps.shape

(8180, 13)

#### 1.2.2 Feature Engineering:
create new features from the existing features. Sometimes it better to create new features than to use the existing features. Also think about how to encode categorical features, normalize features, etc.

In [100]:
# First we transform the categories, content ratings and android versions into numeric labels
encoder = LabelEncoder()
apps['Category_Encoded'] = encoder.fit_transform(apps['Category'])
apps['Content Rating_Encoded'] = encoder.fit_transform(apps['Content Rating'])
apps['Android Ver_Encoded'] = encoder.fit_transform(apps['Android Ver'])

In [101]:
# We also make a new column 'time since last update' out of the existing 'last updated'

current_date = pd.to_datetime('2019-01-01')  # All data is from before 2019
apps['Time Since Last Update'] = (current_date - apps['Last Updated']).dt.days


In [102]:
# We cant work with the App name column, however I think it might be useful to include the length of the app name
apps['App_Name_Length'] = apps['App'].apply(len)

In [103]:
# we will also categorize the prices of the apps into "FREE", "CHEAP" and "EXPENSIVE"
def categorize_price(price):
    if price == 0:
        return "FREE"
    elif 0 < price < 5.0:
        return "CHEAP"
    else:
        return "EXPENSIVE"

# Create the 'Price_Category_Labels' column
apps['Price_Category_Labels'] = apps['Price'].apply(categorize_price)

# Create the 'Price_Category_Encoded' column
apps['Price_Category_Encoded'] = apps['Price_Category_Labels'].map({'FREE': 0, 'CHEAP': 1, 'EXPENSIVE': 2})

In [104]:
# The column we will work with is Installs. I want to know how much the correlations are between Installs and the other numeric columns.

apps.corr(numeric_only=True, method='pearson')['Rating']


Rating                    1.000000
Reviews                   0.055002
Size                      0.037261
Installs                  0.040026
Price                     0.021718
Category_Encoded         -0.041309
Content Rating_Encoded    0.027344
Android Ver_Encoded       0.062483
Time Since Last Update   -0.130443
App_Name_Length           0.143748
Price_Category_Encoded    0.043856
Name: Rating, dtype: float64

In [105]:
# Out of the correlations I see a few high correlations like App_Name_Length and Time Since Last Update (inverse). I don't think any correlation in this list is low enough to exclude out of the dataset for the moment. I will only delete Price, since I think working with price ranges in categories is more useful. I will also drop all the other, non-numeric columns

numeric_columns = apps.select_dtypes(include=['number'])
numeric_columns = numeric_columns.drop(['Price'], axis=1)

# Drop non-numeric columns
apps = apps[numeric_columns.columns]
apps

,Rating,Reviews,Size,Installs,Category_Encoded,Content Rating_Encoded,Android Ver_Encoded,Time Since Last Update,App_Name_Length,Price_Category_Encoded
0,4.1,159,19.0,10000,0,1,14,359,46,0
1,3.9,967,14.0,500000,0,1,14,351,19,0
2,4.7,87510,8.7,5000000,0,1,14,153,50,0
3,4.5,215644,25.0,50000000,0,4,17,207,21,0
4,4.3,967,2.8,100000,0,1,19,195,37,0
...,...,...,...,...,...,...,...,...,...,...
10833,4.0,7,2.6,500,11,1,16,562,13,0
10835,4.5,38,53.0,5000,11,1,16,525,16,0
10836,5.0,4,3.6,100,11,1,16,179,32,0
10838,4.5,114,0.0,1000,3,3,30,1443,29,0


In [106]:
# the last thing I do in feature engineering is transforming the data. This way all the features will have the same range of magnitude.

scaler = StandardScaler()

numeric_columns = ['Reviews', 'Size', 'Installs', 'Category_Encoded', 'Content Rating_Encoded','Android Ver_Encoded', 'Time Since Last Update', 'App_Name_Length', 'Price_Category_Encoded']

apps[numeric_columns] = scaler.fit_transform(apps[numeric_columns])

pd.DataFrame(apps.isna().sum(), columns=['# of missing values']).T


,Rating,Reviews,Size,Installs,Category_Encoded,Content Rating_Encoded,Android Ver_Encoded,Time Since Last Update,App_Name_Length,Price_Category_Encoded
# of missing values,0,0,0,0,0,0,0,0,0,0


#### 1.2.2 Feature Selection:
Remove features that are not useful to improve the performance of the models

In [107]:
# Using f tests for regression and going with the best 75% of features. I think we have enough useful features with 60% as the percentile.

selector = SelectPercentile(score_func=f_regression, percentile=75)
X = selector.fit_transform(apps.select_dtypes(include=np.number), apps.Rating)
best_features = selector.get_support(indices=True)
apps = apps.select_dtypes(include=np.number).iloc[:, best_features]
apps

,Rating,Reviews,Category_Encoded,Android Ver_Encoded,Time Since Last Update,App_Name_Length,Price_Category_Encoded
0,4.1,-0.128607,-2.025024,-0.299978,-0.153015,1.865938,-0.264088
1,3.9,-0.128200,-2.025024,-0.299978,-0.172447,-0.331355,-0.264088
2,4.7,-0.084654,-2.025024,-0.299978,-0.653390,2.191463,-0.264088
3,4.5,-0.020180,-2.025024,0.141409,-0.522223,-0.168592,-0.264088
4,4.3,-0.128200,-2.025024,0.435668,-0.551371,1.133507,-0.264088
...,...,...,...,...,...,...,...
10833,4.0,-0.128683,-0.703577,-0.005720,0.340073,-0.819642,-0.264088
10835,4.5,-0.128668,-0.703577,-0.005720,0.250199,-0.575499,-0.264088
10836,5.0,-0.128685,-0.703577,-0.005720,-0.590235,0.726601,-0.264088
10838,4.5,-0.128629,-1.664630,2.054089,2.480024,0.482457,-0.264088
